The goal of this project is to use SQL on international debt data collected by The World Bank in order to explore, analyze, and understand more about the debt situation across countries. Foundational commands are used as well as a subquery in the where claus, to determine specific facts about international debt status based on the World Bank Database.

Questions are answered such as:

- What is the total amount of debt that is owed by the countries listed?
- Which country owns the maximum amount of debt and what does that amount look like?
- What is the average amount of debt owed by countries across different debt indicators?
- What is the country and indicator name of the biggest amount of debt in a specific indicator?


The dataset from the World Bank looks like this:

|country_name|country_code|indicator_name                                                    |indicator_code|debt       |
|------------|------------|------------------------------------------------------------------|--------------|-----------|
|Afghanistan |AFG         |Principal repayments on external debt, long-term|DT.DIS.DLXF.CD|72894453.7 |
|Afghanistan |AFG         |Principal repayments on external debt, private nonguaranteed|DT.INT.DLXF.CD|53239440.1 |
|Afghanistan |AFG         |Disbursements on external debt, long-term|DT.AMT.BLAT.CD|61739336.9 |
|Afghanistan |AFG         |PPG, official creditors |DT.DIS.BLAT.CD|49114729.4 |
|Afghanistan |AFG         |PPG, private creditors|DT.INT.BLAT.CD|39903620.1 |
|Afghanistan |AFG         |Interest payments on external debt, long-term |DT.AMT.MLAT.CD|39107845   |
|Afghanistan |AFG         |PPG, bilateral |DT.DIS.MLAT.CD|23779724.3 |
|Afghanistan |AFG         |Interest payments on external debt, private nonguaranteed |DT.INT.MLAT.CD|13335820   |
|Afghanistan |AFG         |PPG, official creditors |DT.AMT.OFFT.CD|100847181.9|
|Afghanistan |AFG         |PPG, bonds |DT.DIS.OFFT.CD|72894453.7 |


In [42]:
SELECT COUNT(DISTINCT country_name) AS total_distinct_countries ##how many distinct countries owe debt?
FROM international_debt

,total_distinct_countries
0,124


In [43]:
SELECT DISTINCT(indicator_name)    ##I tried to generate a list of different debt types to understand better.
FROM public.international_debt
ORDER BY indicator_name

,indicator_name
0,"Disbursements on external debt, long-term (DIS..."
1,"Interest payments on external debt, long-term ..."
2,"Interest payments on external debt, private no..."
3,"PPG, bilateral (AMT, current US$)"
4,"PPG, bilateral (DIS, current US$)"
5,"PPG, bilateral (INT, current US$)"
6,"PPG, bonds (AMT, current US$)"
7,"PPG, bonds (INT, current US$)"
8,"PPG, commercial banks (AMT, current US$)"
9,"PPG, commercial banks (DIS, current US$)"


In [44]:
SELECT ROUND(SUM(debt)/1000000,2)  ##Total amount of debt owed by all countries, in millions, rounded to 2 digits. Looks like a huge number.
FROM public.international_debt


,round
0,3079734.49


In [45]:
SELECT country_name, SUM(debt) AS total_debt  ##Country that owed a biggest total debt amount across indicators
FROM public.international_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT (1);

,country_name,total_debt
0,China,2.857935e+11


In [46]:
SELECT indicator_code AS debt_indicator, indicator_name, AVG(debt) AS average_debt
FROM public.international_debt
GROUP BY debt_indicator, indicator_name   ##Average debt amount across indicator and sub-indicators
ORDER BY average_debt DESC
LIMIT 10

,debt_indicator,indicator_name,average_debt
0,DT.AMT.DLXF.CD,"Principal repayments on external debt, long-te...",5.904868e+09
1,DT.AMT.DPNG.CD,"Principal repayments on external debt, private...",5.161194e+09
2,DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS...",2.152041e+09
3,DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1.958983e+09
4,DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1.803694e+09
5,DT.INT.DLXF.CD,"Interest payments on external debt, long-term ...",1.644024e+09
6,DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1.223139e+09
7,DT.INT.DPNG.CD,"Interest payments on external debt, private no...",1.220411e+09
8,DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1.191188e+09
9,DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1.082624e+09


In [47]:
SELECT country_name, indicator_name   ##In the principal repayments, long-term category, China owes the most debt.
FROM public.international_debt    ## A subquery is used, to match specifically the maximum debt in the single category.
WHERE indicator_code = 'DT.AMT.DLXF.CD' AND debt = (SELECT MAX(debt)
													FROM international_debt
													WHERE indicator_code = 'DT.AMT.DLXF.CD');

,country_name,indicator_name
0,China,"Principal repayments on external debt, long-te..."
